## OpenMP: Compilers and Toolchain

Let's recall that an OpenMP program consists of:
1. Compiler directives that generate parallel code
2. Library calls to configure runtime
  a. Need to `#include <omp.h>` to get symbols/functions

* For gcc/g++, OpenMP is built in.  Use the openmp option.
```sh
gcc -fopenmp program.c
````
* For clang, OpenMP is kind of built in.  On MacOSX, it needs help with linking.
```sh
clang -Xpreprocessor -fopenmp -lomp program.c 
```

### Compiler Optimization

The goal is to develop __fast__ and parallel code.  OpenMP makes it parallel. Compilers make it fast.  They perform things like:
  * loop unrolling
  * inline functions
  * branch prediction
  * vectorization
These can effect performance by __large constant factors__.

The degree of optimization is specified at compile time.  It is typical to debug unoptimized code and deploy optimized code.  In gcc/g++:
  * -O0 (default) = Reduce compilation time and make debugging produce the expected results.
  * -O1 = simple optimizations that don’t take a lot of compile time
  * -O2 = rewrite loops, follow jump pointers, inline small functions, no time/space tradeoffs
  * -O3 = vectorize, inline functions, branch prediction
  * and, -g include debugging symbols
  


### Memory Access Patterns (as an example)

* To be efficient, memory access should be __sequential__ and __aligned__
  * Sequential = in memory address order
  * Aligned = to physical boundaries in HW.  Cache lines, memory pages.
  
<img src="https://cvw.cac.cornell.edu/gpu/images/figure8.png" width=512 />

This is a picture from a Cornell workshop, but it makes the point.

* To accomplish this in looping programs
  * choose an iteration order that is sequential in memory
  * allocate data aligned (or hope it happens by default because interfaces are not portable
* Nested loops are a good example
  * Row versus column order can make a big difference.
  * Think of memory as reading a sequential cache line at a time

  
<img src="./images/rowvcol.png" width=512 title="http://akira.ruc.dk/~keld/teaching/IPDC_f10/Slides/pdf4x/4_Performance.4x.pdf" />



### Things we learned in `stencil.c`

1. Compiler optimization matters
2. Different compilers produce different code
  * clang rewrote the loop, gcc didn't
3. Iteration order matters
  * if your compiler doesn't rewrite the loop
  * many things will prevent your compiler from rewriting (more on this later)
4. Caching affects performance (particularly initial runs)
  * Measure on a warm cache.
  * The initial run will often load data into processor cache from memory.